In [ ]:
import site
import subprocess
from pathlib import Path

repo_root = Path.cwd().resolve()
while repo_root != repo_root.parent and not (repo_root / 'pyproject.toml').exists():
    repo_root = repo_root.parent
if not (repo_root / 'pyproject.toml').exists():
    raise RuntimeError('Could not locate repository root with pyproject.toml')

site.addsitedir(str(repo_root / 'python'))
subprocess.run(['cmake', '-S', str(repo_root), '-B', str(repo_root / 'build-release'), '-DCMAKE_BUILD_TYPE=Release'], check=True)
subprocess.run(['cmake', '--build', str(repo_root / 'build-release'), '--target', 'qerasure_python', '-j'], check=True)


In [ ]:
from qerasure import RotatedSurfaceCode, NoiseParams, ErasureSimParams, ErasureSimulator, visualize_erasures


In [ ]:
code = RotatedSurfaceCode(distance=3)
code.draw()


In [ ]:
noise = NoiseParams(p_two_qubit_erasure=0.01, p_erasure_check_error=0.05)


In [ ]:
sim_params = ErasureSimParams(code=code, noise=noise, qec_rounds=10, shots=1000, seed=12345)
simulator = ErasureSimulator(sim_params)


In [ ]:
result = simulator.simulate()


In [ ]:
len(result.erasure_timestep_offsets[0]), len(result.sparse_erasures[0])


In [ ]:
visualize_erasures(result, sim_params, shot_idx=0)
